In [ ]:
import matplotlib.pyplot as plt
from scipy.ndimage.morphology import distance_transform_edt
import tensorflow as tf
import keras.backend as K

import matplotlib.image as mpimg
import numpy as np
import h5py


# check for dataset
input_h5_file = '/nrs/mouselight/Users/base/annotated_neuron/2017-09-25_G-007_consensus-training_sparse_label.h5:volume'
file,dataset = input_h5_file.split(':')
input_raw_f = os.path.join(datafold, file).replace('/', '//')
input_raw_handle = h5py.File(input_raw_f, 'r')[dataset]


test_data = data[0][...,0]
imgplot = plt.imshow(np.max(test_data,axis=0))
# prediction = predict(model, test_data, permute=permute)

input_raw_handle = preload_data(config['data_file'])
input_label_handle = preload_data(config['label_file'])

imgplot = plt.imshow(np.max(input_label_handle[0],axis=0))

##
batch_x = input_raw_handle[0:6]
batch_y = input_label_handle[0:6]
batch_x = np.transpose(batch_x, (0, 2, 3, 4, 1)) # #/z/y/x/ch
# TODO: fix permutation for label volume
batch_y = batch_y[:,:,:,:,None]

def image2dist(inputimage,target_label = 1, mask_thr = 0):
    input_mask = np.greater(inputimage==target_label,mask_thr)
    #skeletonize input mask
    out = []
    for ind in range(input_mask.shape[0]):
        input_data = np.asarray(input_mask[ind,...,0],np.float)
        dist2boundary = distance_transform_edt(input_data<1)
        # out.append(sigmoid(dist2boundary-1)) # at boundary => 0.5
        out.append(dist2boundary) # at boundary => 0.5

    return np.asarray(out,np.float32)[...,None] # extend


#####################################
batch_y2 = image2dist(batch_y)

######

def detection_metric(y_true, y_pred, thr = 1.0,smooth=1.0):
    y_true_f = K.flatten(K.less_equal(y_true,thr))
    y_pred_f = K.flatten(K.less_equal(y_pred,thr))
    y_true_bf = tf.cast(y_true_f,tf.float32)
    y_pred_bf = tf.cast(y_pred_f,tf.float32)
    intersection = K.sum(y_true_bf * y_pred_bf)
    return (2. * intersection + smooth) / (K.sum(y_true_bf) + K.sum(y_pred_bf) + smooth)

def mask_mean_squared_error_loss(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    weight = K.less_equal(y_true_f, 10)
    # weight = K.cast(weight,tf.float32)
    loss = tf.losses.mean_squared_error(y_pred_f, y_true_f,weight,reduction="weighted_sum_over_batch_size")
    return loss

def mask_mean_squared_error_loss_np(y_true,y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    weight = np.less_equal(y_true_f, 10)
    df = y_pred_f-y_true_f
    df = np.power(df,2)
    df_w = df*weight
    return np.mean(df_w),df,df_w,weight
########

def detection_metric_np(y_true, y_pred, thr = 1.0,smooth=1.0):
    y_true_f = np.less_equal(y_true,thr).flatten()
    y_pred_f = np.less_equal(y_pred,thr).flatten()
    y_true_bf = np.asarray(y_true_f,np.float32)
    y_pred_bf = np.asarray(y_pred_f,np.float32)
    intersection = np.sum(y_true_bf * y_pred_bf)
    return (2. * intersection + smooth) / (np.sum(y_true_bf) + np.sum(y_pred_bf) + smooth)

lo_np,df,df_w,weight = mask_mean_squared_error_loss_np(batch_y2,batch_y)
lo = mask_mean_squared_error_loss(batch_y2,batch_y)
dm = detection_metric(batch_y2,np.asarray(batch_y,np.float32), thr = 1.0,smooth=1.0)
dm_np = detection_metric_np(batch_y2,batch_y2, thr = 1.0,smooth=1.0)


sess = tf.Session()
with sess.as_default():
    print(lo.eval())
    print(dm.eval())
print(lo_np)



plt.figure(0)
plt.imshow(np.max(image[...,0],axis=0))
plt.figure(1)
plt.imshow(np.max(label,axis=0))

plt.figure(1)
plt.imshow(np.max(batch_y[1,...,0]==1,axis=0))

plt.figure(2)
plt.imshow(np.max(batch_y2[1,...,0]<=1,axis=0))

# # sitk.Show(sitk.GetImageFromArray(np.swapaxes(label_patch, 2, 0)))
import SimpleITK as sitk
i1 = sitk.GetImageFromArray(np.swapaxes(np.asarray(batch_y[0,...,0],np.uint8),2,0))
sitk.Show(i1)

i2 = sitk.GetImageFromArray(np.swapaxes(np.asarray(batch_y2[0,...,0],np.uint8),2,0))
sitk.Show(i2)

i3 = sitk.GetImageFromArray(np.swapaxes(np.asarray(batch_x[0,...,0],np.uint16),2,0))
sitk.Show(i3)


predict(model, test_data, permute=permute,label_data=label_data)




